# --- Day 7: Camel Cards ---
Your all-expenses-paid trip turns out to be a one-way, five-minute ride in an airship. (At least it's a cool airship!) It drops you off at the edge of a vast desert and descends back to Island Island.

"Did you bring the parts?"

You turn around to see an Elf completely covered in white clothing, wearing goggles, and riding a large camel.

"Did you bring the parts?" she asks again, louder this time. You aren't sure what parts she's looking for; you're here to figure out why the sand stopped.

"The parts! For the sand, yes! Come with me; I will show you." She beckons you onto the camel.

After riding a bit across the sands of Desert Island, you can see what look like very large rocks covering half of the horizon. The Elf explains that the rocks are all along the part of Desert Island that is directly above Island Island, making it hard to even get there. Normally, they use big machines to move the rocks and filter the sand, but the machines have broken down because Desert Island recently stopped receiving the parts they need to fix the machines.

You've already assumed it'll be your job to figure out why the parts stopped when she asks if you can help. You agree automatically.

Because the journey will take a few days, she offers to teach you the game of Camel Cards. Camel Cards is sort of similar to poker except it's designed to be easier to play while riding a camel.

In Camel Cards, you get a list of hands, and your goal is to order them based on the strength of each hand. A hand consists of five cards labeled one of A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2. The relative strength of each card follows this order, where A is the highest and 2 is the lowest.

Every hand is exactly one type. From strongest to weakest, they are:

Five of a kind, where all five cards have the same label: AAAAA

Four of a kind, where four cards have the same label and one card has a different label: AA8AA

Full house, where three cards have the same label, and the remaining two cards share a different label: 23332

Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98

Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432

One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4

High card, where all cards' labels are distinct: 23456
Hands are primarily ordered based on type; for example, every full house is stronger than any three of a kind.

If two hands have the same type, a second ordering rule takes effect. Start by comparing the first card in each hand. If these cards are different, the hand with the stronger first card is considered stronger. If the first card in each hand have the same label, however, then move on to considering the second card in each hand. If they differ, the hand with the higher second card wins; otherwise, continue with the third card in each hand, then the fourth, then the fifth.

So, 33332 and 2AAAA are both four of a kind hands, but 33332 is stronger because its first card is stronger. Similarly, 77888 and 77788 are both a full house, but 77888 is stronger because its third card is stronger (and both hands have the same first and second card).

To play Camel Cards, you are given a list of hands and their corresponding bid (your puzzle input). For example:
```
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
```
This example shows five hands; each hand is followed by its bid amount. Each hand wins an amount equal to its bid multiplied by its rank, where the weakest hand gets rank 1, the second-weakest hand gets rank 2, and so on up to the strongest hand. Because there are five hands in this example, the strongest hand will have rank 5 and its bid will be multiplied by 5.

So, the first step is to put the hands in order of strength:

32T3K is the only one pair and the other hands are all a stronger type, so it gets rank 1.

KK677 and KTJJT are both two pair. Their first cards both have the same label, but the second card of KK677 is stronger (K vs T), so KTJJT gets rank 2 and KK677 gets rank 3.

T55J5 and QQQJA are both three of a kind. QQQJA has a stronger first card, so it gets rank 5 and T55J5 gets rank 4.

Now, you can determine the total winnings of this set of hands by adding up the result of multiplying each hand's bid with its rank (765 * 1 + 220 * 2 + 28 * 3 + 684 * 4 + 483 * 5). So the total winnings in this example are 6440.

**Find the rank of every hand in your set. What are the total winnings?**

In [1]:
from utilities import get_lines

In [2]:
lines = get_lines('input')

In [3]:
len(lines)

1000

In [4]:
from collections import Counter

In [5]:
class Hand:
    card_values = dict(zip('23456789TJQKA',range(13)))
    
    def __init__(self, hand, bid):
        self.hand = hand
        self.num_cards = len(hand)
        self.bid = bid
        
    def __repr__(self):
        return f'{self.hand}'
    
    @classmethod
    def _resolve_conflict(cls, h1, h2):
        for i in range(h1.num_cards):
            if Hand.card_values[h1.hand[i]]>Hand.card_values[h2.hand[i]]:
                return 1
            if Hand.card_values[h1.hand[i]]<Hand.card_values[h2.hand[i]]:
                return -1
        return 0
    
    def __eq__(self, other):
        if self._resolve_conflict(self, other)==0:
            return True
        return False
    
    def __lt__(self, other):
        if self._resolve_conflict(self, other)==-1:
            return True
        return False
    
    def __gt__(self, other):
        if self._resolve_conflict(self, other)==1:
            return True
        return False
    
    @classmethod
    def _get_max_count(cls, h):
        counter = Counter(h.hand)
        return max(counter.items(),
                   key=lambda item: item[1]
                  )[1]
    
    def _is_5_kind(self):
        if len(set(self.hand))==1:
            return True
        return False
    
    def _is_4_kind(self):
        if (len(set(self.hand))==2)&(Hand._get_max_count(self)==4):
            return True
        return False
    
    def _is_full_house(self):
        if (len(set(self.hand))==2)&(Hand._get_max_count(self)==3):
            return True
        return False
    
    def _is_3_kind(self):
        if (len(set(self.hand))==3)&(Hand._get_max_count(self)==3):
            return True
        return False
    
    def _is_2_pair(self):
        if (len(set(self.hand))==3)&(Hand._get_max_count(self)==2):
            return True
        return False
    
    def _is_1_pair(self):
        if len(set(self.hand))==4:
            return True
        return False
    
    def _is_high_card(self):
        if len(set(self.hand))==5:
            return True
        return False
    
    def get_hand_type(self):
        if self._is_5_kind():
            return '5 of a kind'
        if self._is_4_kind():
            return '4 of a kind'
        if self._is_full_house():
            return 'full house'
        if self._is_3_kind():
            return '3 of a kind'
        if self._is_2_pair():
            return '2 pair'
        if self._is_1_pair():
            return '1 pair'
        if self._is_high_card():
            return 'high card'
        return 'no type'
    
    hand_rank = ['high card', '1 pair', '2 pair', '3 of a kind',
                 'full house', '4 of a kind', '5 of a kind'
                ]
    hand_values = dict(zip(hand_rank, range(7)))
    
    def get_hand_value(self):
        return Hand.hand_values[self.get_hand_type()]
    
    def __hash__(self):
        return hash(self.get_hand_value())
    
    @classmethod
    def rank_hands(cls, hand_list):
        sorted_hands = sorted(hand_list, 
                       key=lambda h: Hand.get_hand_value(h)
                      )
        hold_hands = list()
        final_hands = list()
        while len(sorted_hands)>0:
            current_hand = sorted_hands.pop()
            hold_hands.append(current_hand)
            # print(f'current hand {current_hand}')
            print(f'working on rank {current_hand.get_hand_value()}')
            if len(sorted_hands)>0:
                next_hand = sorted_hands[-1]
                # print(f'\tnext hand {next_hand}')
                while next_hand is not None and current_hand.get_hand_value()==next_hand.get_hand_value():
                    next_hand = sorted_hands.pop()
                    hold_hands.append(next_hand)
                    current__hand = next_hand
                    if len(sorted_hands)>0:
                        next_hand = sorted_hands[-1]
                    else:
                        next_hand = None
                        
                       # hold_hands.append(next_hand)
                       # if len(sorted_hands)>0:
                       #     current_hand = next_hand
                       #     print(f'\tcurrent hand {current_hand}')
                       #     if len(sorted_hands)>0:
                       #         next_hand = sorted_hands[-1]
                       #         print(f'\t\tnext hand {next_hand}')
                       #         print(f'len sorted hands: {len(sorted_hands)}')
            hold_hands = sorted(hold_hands)[::-1]
            # print(f'hold hands sorted: {hold_hands}')
            final_hands.extend(hold_hands)
            print(f'len final hands extended: {len(final_hands)}')
            
            hold_hands = list()
        return final_hands[::-1]
                
        
                
        #  now do the tie breaker
        # pop the last element from the sorted list and put it in a temp list
        # peek at the new last element...does it have the same value?
        # if not, take the popped element and put it in the final list
        # if so, pop the new last element from the sorted list and 
        # add it to a temp list
        # continue peeking until the ranks are different
        # sort the temp list by card type and add this to the final list
        # repeat until the end

In [6]:
hands = [Hand(h, int(b)) for h, b in zip([line.split(' ')[0] for line in lines],
             [line.split(' ')[1] for line in lines]
            )
        ]

len(hands)

1000

In [7]:
ranked_hands = Hand.rank_hands(hands)
len(ranked_hands)

working on rank 6
len final hands extended: 1
working on rank 5
len final hands extended: 95
working on rank 4
len final hands extended: 192
working on rank 3
len final hands extended: 384
working on rank 2
len final hands extended: 542
working on rank 1
len final hands extended: 773
working on rank 0
len final hands extended: 1000


1000

In [8]:
print(f'The answer you are looking for is {sum([h.bid*(i+1) for i,h in enumerate(ranked_hands)])}')

The answer you are looking for is 245794640
